In [63]:
# In this exercise you will train a CNN on the FULL Cats-v-dogs dataset
# This will require you doing a lot of data preprocessing because
# the dataset isn't split into training and validation for you
# This code block has all the required inputs
import os
import zipfile
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from numpy.random import uniform

In [64]:
# This code block downloads the full Cats-v-Dogs dataset and stores it as 
# cats-and-dogs.zip. It then unzips it to /tmp
# which will create a tmp/PetImages directory containing subdirectories
# called 'Cat' and 'Dog' (that's how the original researchers structured it)
# If the URL doesn't work, 
# .   visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL
if True:
    !wget --no-check-certificate \
        "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
        -O "/tmp/cats-and-dogs.zip"

    local_zip = '../../Data/tmp/cats-and-dogs.zip'
else:
    local_zip = '../../Data/cats_and_dogs_filtered.zip'
    
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('../../Data/tmp')
zip_ref.close()


--2019-12-24 08:10:40--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.32.66.239, 2600:1404:5800:6aa::e59, 2600:1404:5800:69c::e59
Connecting to download.microsoft.com (download.microsoft.com)|23.32.66.239|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 824894548 (787M) [application/octet-stream]
Saving to: ‘/tmp/cats-and-dogs.zip’

/tmp/cats-and-dogs. 100%[===================>] 786.68M  5.48MB/s    in 2m 24s  

2019-12-24 08:13:05 (5.45 MB/s) - ‘/tmp/cats-and-dogs.zip’ saved [824894548/824894548]



FileNotFoundError: [Errno 2] No such file or directory: '../../Data/tmp/cats-and-dogs.zip'

In [3]:
print(len(os.listdir('../../Data/tmp/PetImages/Cat/')))
print(len(os.listdir('../../Data/tmp/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/PetImages/Cat/'

In [13]:
# Use os.mkdir to create your directories
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'
try:
    os.mkdir('../../Data/tmp/cats-v-dogs')
    os.mkdir('../../Data/tmp/cats-v-dogs/training')
    os.mkdir('../../Data/tmp/cats-v-dogs/testing')
    os.mkdir('../../Data/tmp/cats-v-dogs/training/cats')
    os.mkdir('../../Data/tmp/cats-v-dogs/training/dogs')
    os.mkdir('../../Data/tmp/cats-v-dogs/testing/cats')
    os.mkdir('../../Data/tmp/cats-v-dogs/testing/dogs')
except OSError:
    print('no go jo')

In [31]:
?copyfile

Signature: copyfile(src, dst, *, follow_symlinks=True)
Docstring:
Copy data from src to dst.

If follow_symlinks is not set and src is a symbolic link, a new
symlink will be created instead of copying the file it points to.
File:      /usr/lib/python3.5/shutil.py
Type:      function


In [57]:
# Write a python function called split_data which takes
# a SOURCE directory containing the files
# a TRAINING directory that a portion of the files will be copied to
# a TESTING directory that a portion of the files will be copie to
# a SPLIT SIZE to determine the portion
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# os.path.getsize(PATH) gives you the size of the file
# copyfile(source, destination) copies a file from source to destination
# random.sample(list, len(list)) shuffles a list
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    TRAINING_C = 0
    TESTING_C = 0
    for file_idx in  os.listdir(SOURCE):
            if uniform() <= SPLIT_SIZE:
                copyfile(SOURCE + file_idx, TRAINING + file_idx)
                TRAINING_C += 1
            else:
                copyfile(SOURCE + file_idx, TESTING + file_idx)
                TESTING_C += 1
    print('Number of Test: {}\nNumber of Training: {}\nRation: {}'.format(TESTING_C, TRAINING_C, TESTING_C/(TRAINING_C+TESTING_C)))
# YOUR CODE STARTS HERE
# YOUR CODE ENDS HERE

if False:
    CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
    DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
    TRAINING_DIR = "/tmp/cats-v-dogs/training/"
    TESTING_DIR = "/tmp/cats-v-dogs/testing/"

else:
    CAT_SOURCE_DIR = "../../Data/tmp/PetImages/Cat/"
    DOG_SOURCE_DIR = "../../Data/tmp/PetImages/Dog/"
    TRAINING_DIR = "../../Data/tmp/cats-v-dogs/training/"
    TESTING_DIR = "../../Data/tmp/cats-v-dogs/testing/"
 

    TRAINING_CATS_DIR = TRAINING_DIR + 'cats/'
    TESTING_CATS_DIR =  TESTING_DIR + 'cats/'

    TRAINING_DOGS_DIR = TRAINING_DIR + 'dogs/'
    TESTING_DOGS_DIR = TESTING_DIR + 'dogs/'

split_size = .9
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

Number of Test: 141
Number of Training: 1359
Ration: 0.094
Number of Test: 130
Number of Training: 1370
Ration: 0.08666666666666667


In [40]:
if False:
    print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))
    print(len(os.listdir('/tmp/cats-v-dogs/training/dogs/')))
    print(len(os.listdir('/tmp/cats-v-dogs/testing/cats/')))
    print(len(os.listdir('/tmp/cats-v-dogs/testing/dogs/')))
else:
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/training/cats/')))
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/training/dogs/')))
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/testing/cats/')))
    print(len(os.listdir('../../Data/tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

1500
1500
703
700


In [48]:
# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', input_shape = (150,150,3) ),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation = 'relu' ),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [59]:
#RAINING_DIR = '../../Data/'
train_datagen = ImageDataGenerator(rescale = 1/255.)

train_generator = train_datagen.flow_from_directory(
                        TRAINING_DIR,
                        batch_size = 100,
                        class_mode= 'binary',
                        target_size = (150,150))

#VALIDATION_DIR = #YOUR CODE HERE
test_datagen = ImageDataGenerator(rescale = 1/255.)
test_generator = test_datagen.flow_from_directory(
                        TESTING_DIR,
                        batch_size = 100,
                        class_mode = 'binary',
                        target_size = (150,150)
)



# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 3000 images belonging to 2 classes.
Found 1552 images belonging to 2 classes.


In [62]:
history = model.fit_generator(train_generator,
                              epochs=15,
                              verbose=1,
                              validation_data=test_generator)

# The expectation here is that the model will train, and that accuracy will be > 95% on both training and validation
# i.e. acc:A1 and val_acc:A2 will be visible, and both A1 and A2 will be > .9

Epoch 1/15
30/30 [==============================] - 28s 944ms/step - loss: 1.0531 - acc: 0.5373 - val_loss: 0.6876 - val_acc: 0.4981
Epoch 2/15
30/30 [==============================] - 31s 1s/step - loss: 0.7135 - acc: 0.6053 - val_loss: 0.6179 - val_acc: 0.6405
Epoch 3/15
30/30 [==============================] - 31s 1s/step - loss: 0.6472 - acc: 0.6670 - val_loss: 0.5631 - val_acc: 0.7010
Epoch 4/15
30/30 [==============================] - 31s 1s/step - loss: 0.6028 - acc: 0.6763 - val_loss: 0.5964 - val_acc: 0.6765
Epoch 5/15
30/30 [==============================] - 33s 1s/step - loss: 0.5509 - acc: 0.7170 - val_loss: 0.5590 - val_acc: 0.6920
Epoch 6/15
30/30 [==============================] - 31s 1s/step - loss: 0.5406 - acc: 0.7440 - val_loss: 0.4454 - val_acc: 0.8041
Epoch 7/15
30/30 [==============================] - 30s 1s/step - loss: 0.4829 - acc: 0.7693 - val_loss: 0.4358 - val_acc: 0.8215
Epoch 8/15
30/30 [==============================] - 31s 1s/step - loss: 0.4194 - acc: 0

In [ ]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

In [ ]:
# Here's a codeblock just for fun. You should be able to upload an image here 
# and have it classified without crashing

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(# YOUR CODE HERE))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a dog")
  else:
    print(fn + " is a cat")